In [68]:
import fs_structs
from pathlib import Path

In [69]:
CURRO = True

In [70]:
if CURRO:
    NS_PATH ="G:\\fs_namespaces\\prueba"
    TMP_PATH = "G:\\fs_namespaces\\prueba\\tmp"

    NS_PATH ="C:\\fs_namespaces\\prueba"
    TMP_PATH = "C:\\fs_namespaces\\prueba\\tmp"
else:
    NS_PATH = "/home/augusto/python/notebooks/fs_namespaces/prueba"
    TMP_PATH = "/home/augusto/python/notebooks/fs_namespaces/tmp"

In [71]:
ns = fs_structs.structs.FSNamespace(NS_PATH, TMP_PATH)
ns.clear()

In [72]:
ns.names()

[]

In [73]:
ns.names_types()

[]

In [74]:
ns.type("a")

In [75]:
# fast= True no utiliza el directorio tmp para nada.
# En este caso las operaciones set y pop no son atómicas.
# Con fast = False (por defecto) las operaciones del y pop
# son atómicas y se utiliza el directorio tmp. 
# En teoría si tmp apunta al ssd local, debería ser más rápido y
# se debería garantizar igualmente la atomicidad de las operaciones y,
# por lo tanto, debería ser robusto para que sea distribuido.  

a = ns.udict("a", fast=False)

In [76]:
ns.names_types()

[('a', 'ud')]

In [77]:
# lists unhashable in python
# dicts currently not supported as keys. Error with filenames including : in windows. To do.

a["1"] = 3
a[1] = 10
a["go"] = "hola"
a[(1,2)] = [1, 2 , (1,4,5), "kk"]
a["kk"] = {"a": 1, "b":[1,2]}

In [78]:
assert a["1"] == 3
assert a[1] == 10
assert a["go"] == "hola"
assert a[(1,2)] == [1, 2 , (1,4,5), "kk"]
assert a["kk"] == {"a": 1, "b":[1,2]}
assert a.keys() == ['1', 'go', 'kk', (1, 2), 1]
assert a.items() == [('1', 3), ('go', 'hola'), ('kk', {'a': 1, 'b': [1, 2]}),((1, 2), [1, 2, (1, 4, 5), 'kk']),(1, 10)]
assert len(a) == 5

In [79]:
# a y a_eq apuntan al mismo objeto.
# Si los directorios tmp no coinciden no debería ser un problema.
# La atomicidad de las operaciones set y pop deberían garantizarse igualmente.

a_eq = fs_structs.structs.FSUDict(Path(NS_PATH).resolve() / "ud_a", TMP_PATH, fast=True)

In [80]:
assert a_eq["1"] == 3
assert a_eq[1] == 10
assert a_eq["go"] == "hola"
assert a_eq[(1,2)] == [1, 2 , (1,4,5), "kk"]
assert a_eq["kk"] == {"a": 1, "b":[1,2]}
assert a_eq.keys() == ['1', 'go', 'kk', (1, 2), 1]
assert a_eq.items() == [('1', 3), ('go', 'hola'), ('kk', {'a': 1, 'b': [1, 2]}),((1, 2), [1, 2, (1, 4, 5), 'kk']),(1, 10)]
assert len(a_eq) == 5

In [81]:
a[{"adad":1233, (1,3,4):[1,2,3,4], 3:1}] = 1234

In [82]:
assert a[{"adad":1233, (1,3,4):[1,2,3,4], 3:1}] == 1234

In [83]:
del a["go"]

In [84]:
assert not "go" in a.keys()
assert 1 in a.keys()

In [85]:
a.update([(x,x) for x in range(100)])

In [86]:
for i in range(100):
    assert a[i] == i

In [87]:
a = ns.variable("a")
a.clear()
assert len(a) == 0

In [88]:
y = ns.list("y")
y.clear()
#y.data.fast = False

lst = []

In [89]:
ns.names_types()

[('y', 'li'), ('a', 'ud')]

In [90]:
_ = [y.append(i) for i in range(1000)]
_ = [lst.append(i) for i in range(1000)]

In [91]:
# length and values

assert len(y) == len(lst)
assert sum(y.values()) == sum(lst)

In [92]:
# order
assert y.values() == lst

In [93]:
y.clear()
y.extend(lst)

In [94]:
assert len(y) == len(lst)
assert sum(y.values()) == sum(lst)

In [95]:
# order
assert y.values() == lst

In [96]:
k = y.keys()
v = y.values()
errors = []
for i in range(len(lst)):
    if v[i] != lst[i]:
        #print(i)
        errors.append(i)

len(errors)

0

In [97]:
len([k[i+1][0] - k[i][0] for i in range(len(v)-1) if (k[i+1][0] - k[i][0]) <= 0])

0

In [98]:
y[0] = -1
lst[0] = -1

y[5] = 12
lst[5] = 12

In [99]:
assert y.values() == lst

In [100]:
assert y.values()[:10] == lst[:10]

In [101]:
_ = [y.pop(0) for i in range(100)]
_ = [lst.pop(0) for i in range(100)]

In [102]:
assert y.values() == lst

In [103]:
_ = [y.insert(0, i) for i in range(100)]
_ = [lst.insert(0, i) for i in range(100)]

In [104]:
assert len(y) == len(lst)
assert sum(y.values()) == sum(lst)

In [105]:
assert y.values() == lst

In [106]:
_ = [y.insert(i, i) for i in range(10)]
_ = [lst.insert(i, i) for i in range(10)]

In [107]:
assert len(y) == len(lst)
assert sum(y.values()) == sum(lst)

In [108]:
assert y.values() == lst

In [109]:
ns.names_types()

[('y', 'li'), ('a', 'ud')]

In [110]:
ns.namespace("cositas")

In [111]:
ns.names_types()

[('y', 'li'), ('cositas', 'ns'), ('a', 'ud')]

In [112]:
aa = ns.cositas.udict("a")

In [113]:
aa["b"] = 1

In [114]:
aa["b"]

1

In [115]:
ns.cositas.a["b"] =2

In [116]:
ns.cositas.a["c"] = 3

In [117]:
ns.cositas.udict("a")["b"]

2

In [118]:
isinstance(ns.cositas, fs_structs.structs.FSNamespace)

True